<a href="https://colab.research.google.com/github/UlviyyaMammadova24/APP117FTask/blob/main/Layihe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets accelerate

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch


train_df = pd.read_csv('training.csv')
val_df = pd.read_csv('validation.csv')


train_df = train_df.rename(columns={'label': 'labels'})
val_df = val_df.rename(columns={'label': 'labels'})


train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):

    result = tokenizer(examples["text"], padding="max_length", truncation=True)
    result["labels"] = examples["labels"]
    return result


print("Mətnlər emal olunur...")
tokenized_train = train_ds.map(tokenize_function, batched=True, remove_columns=train_ds.column_names)
tokenized_val = val_ds.map(tokenize_function, batched=True, remove_columns=val_ds.column_names)

tokenized_train.set_format("torch")
tokenized_val.set_format("torch")


device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6).to(device)


training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="no",
    fp16=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

print(f"Öyrətmə başladı (Cihaz: {device})...")
trainer.train()


model.save_pretrained("emotion_model_final")
tokenizer.save_pretrained("emotion_model_final")
print("Model 'emotion_model_final' qovluğuna uğurla yadda saxlanıldı!")

In [ ]:

!pip install gradio -q

import gradio as gr
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch


model_path = "emotion_model_final"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)


emotions = {
    0: "Sadness (Kədər) 😢",
    1: "Joy (Sevinc) 😊",
    2: "Love (Sevgi) ❤️",
    3: "Anger (Qəzəb) 😠",
    4: "Fear (Qorxu) 😨",
    5: "Surprise (Təəccüb) 😲"
}

def predict_emotion(text):
    if not text.strip():
        return "Zəhmət olmasa bir cümlə yazın."

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
        prediction = torch.argmax(probabilities, dim=1).item()
        confidence = probabilities[0][prediction].item() * 100

    result = emotions.get(prediction, "Naməlum")
    return f"Nəticə: {result} ({confidence:.2f}%)"


demo = gr.Interface(
    fn=predict_emotion,
    inputs=gr.Textbox(lines=2, placeholder="İngiliscə bir cümlə yazın..."),
    outputs="text",
    title="Deep Learning Emotion Classifier",
    description="Colab-da öyrədilmiş modelin canlı testi",
    examples=[
        ["I lost something very precious to me today."],
        ["I am so excited about our upcoming trip!"],
        ["I feel a bit nervous about the presentation."]
    ]
)


demo.launch(share=True, debug=True)